In [1]:
from datasets import load_dataset, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import numpy as np
import pandas as pd

# Load the CSV file into a Pandas DataFrame
file_path = '/kaggle/input/cort-data-class-dataset/dataclass_df_train.csv'
df = pd.read_csv(file_path)

# Prepare the dataset for Hugging Face
def preprocess_data(df):
    # Rename columns for consistency
    df.columns = ['text', 'label']
    df['label'] = df['label'].astype(int)
    return Dataset.from_pandas(df)

# Preprocess the dataset
dataset = preprocess_data(df)

# Split dataset into train and validation sets (80-20 split)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']

# Load the tokenizer for CodeBERT (RoBERTa-base)
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Load CodeBERT model for sequence classification
model = RobertaForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=2)

# Define evaluation metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels, preds, average='macro')
    f1 = f1_score(labels, preds, average='macro')
    accuracy = accuracy_score(labels, preds)
    return {'accuracy': accuracy, 'f1': f1, 'precision': precision, 'recall': recall}

# Define the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

## 0e92fed1c003f0fdabc054357fbb76beb18ff2dc

# Train the model
trainer.train()

# Save the best model
trainer.save_model('./best_codebert_model')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

...

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.574800,0.537960,0.695833,0.647336,0.818408,0.674107
2,0.273400,0.341016,0.866667,0.865083,0.869991,0.863281
3,0.278500,0.347508,0.870833,0.866950,0.891255,0.863281
4,0.087000,0.287373,0.925000,0.924364,0.926922,0.922991
5,0.147300,0.284142,0.925000,0.924576,0.925175,0.924107
6,0.078200,0.474035,0.916667,0.916522,0.916227,0.917969
7,0.072600,0.476551,0.895833,0.893008,0.914912,0.888951
8,0.095700,0.356880,0.929167,0.928889,0.928681,0.929129
9,0.061500,0.355556,0.933333,0.932768,0.935370,0.931362
10,0.001800,0.445136,0.920833,0.920223,0.922177,0.919085


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

In [2]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, matthews_corrcoef
import numpy as np
from datasets import Dataset

# Load your test set (assuming it's a CSV file)
test_file_path = '/kaggle/input/cort-data-class-dataset/dataclass_df_test.csv'
test_df = pd.read_csv(test_file_path)

# Prepare the test dataset
def preprocess_test_data(df):
    df.columns = ['text', 'label']
    df['label'] = df['label'].astype(int)
    return Dataset.from_pandas(df)

test_dataset = preprocess_test_data(test_df)

# Tokenize the test dataset
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Make predictions
predictions = trainer.predict(test_dataset)

# Get predicted labels and true labels
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Calculate evaluation metrics
accuracy = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='macro')
precision = precision_score(labels, preds, average='macro')
recall = recall_score(labels, preds, average='macro')
mcc = matthews_corrcoef(labels, preds)

# Display the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Macro: {f1:.4f}")
print(f"Precision Macro: {precision:.4f}")
print(f"Recall Macro: {recall:.4f}")
print(f"MCC: {mcc:.4f}")


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Accuracy: 0.9133
F1 Macro: 0.9131
Precision Macro: 0.9170
Recall Macro: 0.9133
MCC: 0.8303


In [6]:
preds[4]

0